In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

from delta import *

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

25/04/12 10:54:16 WARN Utils: Your hostname, edrik-Inspiron-15-3520 resolves to a loopback address: 127.0.1.1; using 192.168.0.48 instead (on interface wlp2s0)
25/04/12 10:54:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/edrik/dev/projeto-eng-dados/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/edrik/.ivy2/cache
The jars for the packages stored in: /home/edrik/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-76b3fbee-c7aa-47df-ab80-92ea707976d6;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 168ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [3]:
spark

In [4]:
spark.sql(
  """
  CREATE TABLE carro_delta (id INT, placa STRING) USING delta
  """
)

DataFrame[]

In [5]:
spark.sql("select * from carro_delta").show()

25/04/12 10:55:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+-----+
| id|placa|
+---+-----+
+---+-----+



In [6]:
from delta.tables import DeltaTable

carro = DeltaTable.forPath(spark, "./spark-warehouse/carro_delta")

In [7]:
carro.history().show()

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+
|      0|2025-04-12 10:54:...|  NULL|    NULL|CREATE TABLE|{partitionBy -> [...|NULL|    NULL|     NULL|       NULL|  Serializable|         true|              {}|        NULL|Apache-Spark/3.5....|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+



In [8]:
spark.sql("INSERT INTO carro_delta VALUES (1, 'XYZ1J34'), (2, 'RLC5B93'), (3, 'ABV1V23')")

DataFrame[]

In [9]:
spark.sql("select * from carro_delta").show()

+---+-------+
| id|  placa|
+---+-------+
|  2|RLC5B93|
|  3|ABV1V23|
|  1|XYZ1J34|
+---+-------+



In [10]:
carro.history().show(truncate=False)

+-------+-----------------------+------+--------+------------+----------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                           |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+------------+----------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|1  

In [11]:
spark.sql(
    """
    alter table carro_delta add column marca STRING, modelo STRING, ano INT
    """
)

DataFrame[]

In [12]:
spark.sql(
    """
    select * from carro_delta
    """
).show()

+---+-------+-----+------+----+
| id|  placa|marca|modelo| ano|
+---+-------+-----+------+----+
|  2|RLC5B93| NULL|  NULL|NULL|
|  3|ABV1V23| NULL|  NULL|NULL|
|  1|XYZ1J34| NULL|  NULL|NULL|
+---+-------+-----+------+----+



In [13]:
spark.sql(
    """
    update carro_delta set marca = 'Renault', modelo = 'Sandero', ano = 2021 where id = 1
    """
)

DataFrame[num_affected_rows: bigint]

In [14]:
spark.sql(
    """
    select * from carro_delta
    """
).show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  1|XYZ1J34|Renault|Sandero|2021|
|  2|RLC5B93|   NULL|   NULL|NULL|
|  3|ABV1V23|   NULL|   NULL|NULL|
+---+-------+-------+-------+----+



In [15]:
DeltaTable.isDeltaTable(spark, "spark-warehouse/carro_delta")

True

In [16]:
spark.sql('select * from carro_delta').show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  1|XYZ1J34|Renault|Sandero|2021|
|  2|RLC5B93|   NULL|   NULL|NULL|
|  3|ABV1V23|   NULL|   NULL|NULL|
+---+-------+-------+-------+----+



In [17]:
spark.sql('update carro_delta set marca="GM", modelo="tracker", ano=2020 where id = 2    ')

DataFrame[num_affected_rows: bigint]

In [18]:
spark.sql('update carro_delta set marca="Ford", modelo="EcoSport", ano=2022 where id = 3    ')

DataFrame[num_affected_rows: bigint]

In [19]:
spark.sql('select * from carro_delta').show()

+---+-------+-------+--------+----+
| id|  placa|  marca|  modelo| ano|
+---+-------+-------+--------+----+
|  1|XYZ1J34|Renault| Sandero|2021|
|  3|ABV1V23|   Ford|EcoSport|2022|
|  2|RLC5B93|     GM| tracker|2020|
+---+-------+-------+--------+----+



In [20]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [21]:
spark.sql('describe HISTORY carro_delta').show(truncate=False);

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                                                                                                           